In [65]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# load the Tiny Shakespeare dataset
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

In [66]:
# get the text from the dataset
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')

# create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# numerically represent the characters
text_as_int = np.array([char2idx[c] for c in text])

# create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [67]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [68]:
# batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [69]:
import tensorflow as tf

# Length of the vocabulary
vocab_size = len(vocab)  # make sure vocab is defined with your vocabulary

# The embedding dimension
embedding_dim = 256  # or your desired embedding dimension

# Number of RNN units
rnn_units = 1024

# Define the batch size
BATCH_SIZE = 64  # or any desired batch size

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=embedding_dim),
        tf.keras.layers.LSTM(rnn_units, 
                             return_sequences=True, 
                             stateful=True, 
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# Build the model
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)



In [70]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [71]:
import os
import tensorflow as tf

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Name of the checkpoint files with the required extension
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

# Create the ModelCheckpoint callback with the corrected filepath
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    verbose=1  # Optional for more output during training
)

# Train the model
EPOCHS =10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 2.9411
Epoch 1: saving model to ./training_checkpoints\ckpt_1.weights.h5
155/155 ━━━━━━━━━━━━━━━━━━━━ 498s 3s/step - loss: 2.9382
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 1.9285
Epoch 2: saving model to ./training_checkpoints\ckpt_2.weights.h5
155/155 ━━━━━━━━━━━━━━━━━━━━ 556s 4s/step - loss: 1.9280
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 1.6607
Epoch 3: saving model to ./training_checkpoints\ckpt_3.weights.h5
155/155 ━━━━━━━━━━━━━━━━━━━━ 524s 3s/step - loss: 1.6605
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 1.5201
Epoch 4: saving model to ./training_checkpoints\ckpt_4.weights.h5
155/155 ━━━━━━━━━━━━━━━━━━━━ 552s 4s/step - loss: 1.5200
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 1.4358
Epoch 5: saving model to ./training_checkpoints\ckpt_5.weights.h5
155/155 ━━━━━━━━━━━━━━━━━━━━ 577s 4s/step - loss: 1.4357
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - lo

In [89]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)  # Shape: (1, len(input))

    text_generated = []

    for i in range(num_generate):
        predictions = model(input_eval)
        
        # Take the last prediction in the sequence
        predictions = predictions[:, -1, :]  # Shape becomes (1, vocab_size)

        # Sample the predicted character
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"QUEEN: So, lets end this"))


QUEEN: So, lets end this untreth the madequers to the life; O comfort, my lord;
they will sure you, let him dralt have burdell'd.

FRIAR LAURENCE:
My lord; we woo. Hark, being soft by entime,
If he wandon wife,
Enumile in the flairly infriat,'
Or that instrument peace,
Twomall our purpose; he's these holy brows:
They I was married lagger wrents and cloudy day is see!
What tirld the slight on thy fancy:
Show me his right, all asks in manively love,
And patied want that words in him that have nothing him
does high first than goods
Our players of you talk on our discorrown.
The promicia larkless plantage of her grave!

WARWICK:
Here's oon to the villain lookings thou! Cominius.'
Why, that doth civil black way
Upen these falls or sword, I remain phavost: where ho!
Come, you shall be quick aid or better vinity.
Now, sir, what he was denied majesty:
Both prepared:
'Where is I anto the Laft and by rove.
Yet still be a sin's watch, and thou wilt sent them.
'Will you say and hear me fall.

ESCA

In [96]:
# Save the entire model (architecture + weights) to a file
model.save('fomlfinal.h5')


In [99]:
from tensorflow.keras.models import load_model

model = load_model('fomlfinal.h5', compile=False)


In [102]:
# Compile the model with appropriate settings
model.compile(
    optimizer='adam',  # Replace with your preferred optimizer
    loss='sparse_categorical_crossentropy',  # Replace with your specific loss if needed
    metrics=['accuracy']  # Add any metrics you wish to track
)

In [103]:
# Print the model structure
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_18 (Embedding)             │ (64, 100, 256)              │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (64, 100, 1024)             │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (64, 100, 65)               │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)